In [1]:
pip install gym==0.10

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\dcnlab\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [2]:
import copy
from sre_constants import SUCCESS
import numpy as np
import torch
import random
from matplotlib import pylab as plt
import gym
#from gym import wrappers 
from IPython.display import clear_output

L1 = 2
L2 = 70
L3 = 40
L4 = 3

model = torch.nn.Sequential(
    torch.nn.Linear(L1, L2),
    torch.nn.ReLU(),
    torch.nn.Linear(L2, L3),
    torch.nn.ReLU(),
    torch.nn.Linear(L3,L4)
)

model2 = copy.deepcopy(model) #target network
model2.load_state_dict(model.state_dict()) #將原始Q網路中的參數複製給目標網路
loss_fn = torch.nn.MSELoss()
learning_rate = 1e-3
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

gamma = 0.9
epsilon = 0.6

env = gym.envs.make('MountainCar-v0')
env = env.unwrapped
#env = wrappers.Monitor(env,"C:\testml\video",video_callable=lambda count:count%10==0)
#env.Monitor.start('C:\testml\video',video_callable=lambda count: count % 10 == 0)

from collections import deque
epochs = 1000 #訓練次數
losses = []
step_num = []
mem_size = 1000 
batch_size = 64 #batch_size
replay = deque(maxlen=mem_size) #replay buffer
max_moves = 1000 #每場遊戲最多可以走幾步
sync_freq = 200
success_flag=0
success_num=0
j=0
for i in range(epochs):
    state1_ = env.reset()
    state1 = torch.from_numpy(state1_).float()
    status = 1
    success_flag=0
    mov = 0     #記錄移動的步數
    while(status == 1): 
        j += 1
        mov += 1
        qval = model(state1) #輸出各動作的Q值
        #print(qval,i)
        qval_ = qval.data.numpy()
        if (random.random() < epsilon):
            action_ = np.random.randint(0,2)
        else:
            action_ = np.argmax(qval_)
        state2_, reward , done , info= env.step(action_)
        #env.render() 
        #print(state2_)
        """
        if reward != -1:
            #step_num.append(mov)
            success_flag=1
            success_num+=1
            reward=200
        """

        if mov == 700 :
            reward = reward -0.5
        if state2_[0] >= -0.4 :
            reward +=  0.5 + state2_[0]
        if state2_[1] >= 0.02 :
            reward +=   0.2 + state2_[1]

        if state2_[0] > 0.5 and success_flag == 0:
            status = 0
            #reward += 20
            success_num+=1
            step_num.append(mov)
            success_flag=1
        
        #if state2_[0] > 0.5:
                #reward = 5
        #elif state2_[0] <= -0.6:
            #reward = -1 + state2_[0]
        state2 = torch.from_numpy(state2_).float()
        #print(i)
        exp = (state1, action_, reward, state2, done)
        replay.append(exp)
        state1 = state2 
        #print(state1)
        #print(len(replay),batch_size)     

        if len(replay) > batch_size:
            minibatch = random.sample(replay, batch_size)
            state1_batch_ = torch.cat([s1 for (s1,a,r,s2,d) in minibatch])
            action_batch = torch.Tensor([a for (s1,a,r,s2,d) in minibatch])
            reward_batch = torch.Tensor([r for (s1,a,r,s2,d) in minibatch])
            state2_batch_ = torch.cat([s2 for (s1,a,r,s2,d) in minibatch])
            done_batch = torch.Tensor([d for (s1,a,r,s2,d) in minibatch])
            state1_batch = torch.reshape(state1_batch_, (64, 2))
            state2_batch = torch.reshape(state2_batch_, (64, 2))

            Q1 = model(state1_batch) 
            with torch.no_grad():     
                Q2 = model2(state2_batch) 
            Y = reward_batch + gamma * ((1-done_batch) * torch.max(Q2,dim=1)[0])
            X = Q1.gather(dim=1,index=action_batch.long().unsqueeze(dim=1)).squeeze()
            loss = loss_fn(X, Y.detach())

            optimizer.zero_grad()
            loss.backward()
            optimizer.step() 
            losses.append(loss.item())    

            if j % sync_freq == 0:
                model2.load_state_dict(model.state_dict())
            
        if reward == 200 or mov > max_moves:
            status = 0 
            mov = 0
        #losses.append(loss.item())  
    if epsilon > 0.1: 
        epsilon -= (1/epochs) 
    """if i % 100 == 0:
        if i!=0 and i%1000 == 0 :
            print(i,loss)
        else:
            print(i)"""
    if success_flag == 0:
        step_num.append(1000)
    #print('\r')
    print(i, loss.item(),success_num)
    #print('\r\b')
#env.close()
#test_model(model)
plt.figure(figsize=(10,7))
plt.plot(losses)
plt.xlabel("Steps",fontsize=11)
plt.ylabel("Loss",fontsize=11)

plt.figure(figsize=(10,7))
plt.plot(step_num)
plt.xlabel("epochs",fontsize=11)
plt.ylabel("action number",fontsize=11)




c:\Users\dcnlab\AppData\Local\Programs\Python\Python39\lib\site-packages\gym\envs\registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
0 0.30835744738578796 0
1 0.09020998328924179 0
2 0.00821461621671915 0
3 0.0028386684134602547 0
4 0.004311225842684507 0
5 0.0045865983702242374 0
6 0.011486680246889591 0
7 0.0009847467299550772 0
8 0.004921761341392994 0
9 0.00419735349714756 0
10 0.0034123959485441446 1
11 0.003547104075551033 1
12 0.0119120879098773 1
13 0.002928974572569132 1
14 0.004746329504996538 1
15 0.007065182086080313 1
16 0.004477425012737513 1
17 0.009489580057561398 2
18 0.0035636648535728455 2
19 0.006088813301175833 2
20 0.0016932273283600807 2
21 0.0014752235729247332 2
22 0.00639058742672205 3
23 0.0031330047640949488 3
24 0.003250724170356989 4
25 0.001557658426463604 5
26 0.002768258797004819 6
27 0.004827498458325863 7
28 0.0031373128294944763 8
29 0.0038225562311708927 9
30 0.002899018581956625 9
31 0.005487150512635708 10
32 0.0025238837115466595 11
33 0.003737202612683177 12
34 0.002481953240931

KeyboardInterrupt: 

In [12]:
"""def test_model(model):
  i = 0
  env = gym.envs.make('MountainCar-v0')
  env = env.unwrapped
  for i in range(10):
    state3_ = env.reset()  
    env.render()
    state3 = torch.from_numpy(state3_).float()
    status = 1
    mov = 0 #記錄移動的步數，初始化為0
    while(status == 1):    
      mov += 1
      qval = model(state3)
      qval_ = qval.data.numpy()
      action_ = np.argmax(qval_) 
      print(action_)
      observation, reward, done, info = env.step(action_)
      state3=torch.from_numpy(observation).float()
      #print(action_,reward,done)
      #print(observation)
      env.render()
      if mov >= 1000:
        status=0
      #time.sleep(0.02)
      #env.render()    # 更新并渲染画面
    print(i)
  env.close()"""

In [23]:

#env = gym.envs.make('MountainCar-v0')
#env = env.unwrapped
test_s=0
for i in range(20):
    state3_ = env.reset()  
    env.render()
    state3 = torch.from_numpy(state3_).float()
    status = 1
    mov = 0 #記錄移動的步數，初始化為0
    while(status == 1):    #一次训练最大运行次数
        mov += 1
        qval = model(state3)
        qval_ = qval.data.numpy()
        action_ = np.argmax(qval_) 
        #print(action_)
        observation, reward, done, info = env.step(action_)
        state3=torch.from_numpy(observation).float()
        #print(action_,reward,done)
        #print(observation)
        env.render()
        if observation[0] > 0.5 :
            status = 0
            test_s += 1
        if mov >= 1000:
            status=0
        #time.sleep(0.02)
        #env.render()    # 更新并渲染画面
    print(i)
print(test_s)
#env.close()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
17


In [36]:
#test_model(model)

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


Exception ignored in: <function Viewer.__del__ at 0x00000178E0F0DCA0>
Traceback (most recent call last):
  File "c:\Users\dcnlab\AppData\Local\Programs\Python\Python39\lib\site-packages\gym\envs\classic_control\rendering.py", line 143, in __del__
    self.close()
  File "c:\Users\dcnlab\AppData\Local\Programs\Python\Python39\lib\site-packages\gym\envs\classic_control\rendering.py", line 62, in close
    self.window.close()
  File "c:\Users\dcnlab\AppData\Local\Programs\Python\Python39\lib\site-packages\pyglet\window\win32\__init__.py", line 332, in close
    super(Win32Window, self).close()
  File "c:\Users\dcnlab\AppData\Local\Programs\Python\Python39\lib\site-packages\pyglet\window\__init__.py", line 858, in close
    app.windows.remove(self)
  File "c:\Users\dcnlab\AppData\Local\Programs\Python\Python39\lib\_weakrefset.py", line 114, in remove
    self.data.remove(ref(item))
KeyError: <weakref at 0x00000178EE6383B0; to 'Win32Window' at 0x00000178EE635BB0>


1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
